[![GitHub issues by-label](https://img.shields.io/github/issues-raw/pfebrer/sisl/BandsPlot?style=for-the-badge)](https://github.com/pfebrer/sisl/labels/BandsPlot)

    
    
BandsPlot 
=========

In [ ]:
import sisl

# This is just for convenience to retreive files
siesta_files = sisl.get_environ_variable("SISL_FILES_TESTS") / "siesta"

Let's get a bands_plot from a `.bands` file

In [ ]:
bands_plot = sisl.get_sile(
    siesta_files / "SrTiO3" / "unpolarized" / "SrTiO3.bands"
).plot()

and see what we've got:

In [ ]:
bands_plot

## Getting the bands that you want

By default, `BandsPlot` gives you **the 15 bands below and above 0 eV** (which is interpreted as the fermi level). 

There are two main ways to specify the bands that you want to display: `Erange` and `bands_range`.

As you may have guessed, `Erange` specifies the energy range that is displayed:

In [ ]:
bands_plot.update_inputs(Erange=[-10, 10])

while with `bands_range` you can actually indicate the indices.

However, note that **`Erange` has preference over `bands_range`**, therefore you need to set `Erange` to `None` if you want the change to take effect.

In [ ]:
bands_plot.update_inputs(bands_range=[6, 15], Erange=None)

If your fermi level is not correctly set or you want a different energy reference, you can provide a value for `E0` to specify where your 0 should be and the bands to display will be automatically calculated from that. 

However, if you want to update `E0` after the plot has been build and you want `BandsPlot` to recalculate the bands for you you will need to set `Erange` and `bands_range` to `None` again.

In [ ]:
bands_plot.update_inputs(E0=-10, bands_range=None, Erange=None)

Notice how only 25 bands are displayed now: the only 10 that are below 0 eV (there are no lower states) and 15 above 0 eV.

In [ ]:
# Set them back to "normal"
bands_plot = bands_plot.update_inputs(E0=0, bands_range=None, Erange=None)

Notice that in spin polarized bands, **you can select the spins to display using the `spin` setting**, just pass a list of spin components (e.g. `spin=[0]`).

Individual bands in legend
--------------------------

Usually, showing all bands individually in the legend would be too messy. However, you might want to do it so that you can interactively hide show certain bands. If that is the case, you can set `group_legend` to `False`:

In [ ]:
bands_plot.update_inputs(group_legend=False)

In [ ]:
bands_plot = bands_plot.update_inputs(group_legend=True)

## Bands styling

If all you want is to change the color and width of the bands, there's one simple solution: use the `bands_style` input to tweak the line styles.

Let's show them in red:

In [ ]:
bands_plot.update_inputs(bands_style={"color": "red"})

And now in green but also make them wider:

In [ ]:
bands_plot.update_inputs(bands_style={"color": "green", "width": 3})

If you have spin polarized bands, `bands_style` will tweak the colors for the first spin channel, while the second one can be tuned with `spindown_style`.

Finally, you can pass functions to the keys of `bands_style` to customize the styles on a band basis, or even on a point basis. The functions should accept `data` as an argument, which will be an `xarray.Dataset` containing all the bands data. It should then return a single value or an array of values. It is best shown with examples. Let's create a function just to see what we receive as an input:

In [ ]:
def color(data):
    """Dummy function to see what we receive."""
    print(data)
    return "green"


bands_plot.update_inputs(bands_style={"color": color})

So, you can see that we receive a `Dataset`. The most important variable is `E`, which contains the energy (that depends on `k` and `band`). Let's now play with it to do some custom styling:
- The **color** will be determined by **the slope of the band**.
- We will plot **bands that are closer to the fermi level bigger** because they are more important.

In [ ]:
def gradient(data):
    """Function that computes the absolute value of dE/dk.

    This returns a two dimensional array (gradient depends on k and band)
    """
    return abs(data.E.differentiate("k"))


def band_closeness_to_Ef(data):
    """Computes how close one band is to the fermi level.

    This returns a one dimensional array (distance depends only on band)
    """
    dist_from_Ef = abs(data.E).min("k")

    return (1 / dist_from_Ef**0.4) * 5


# Now we are going to set the width of the band according to the distance from the fermi level
# and the color according to the gradient. We are going to set the colorscale also, instead of using
# the default one.
bands_plot.update_inputs(
    bands_style={"width": band_closeness_to_Ef, "color": gradient},
    colorscale="temps",
    Erange=[-10, 10],
)

You can see that by providing callables the possibilities are endless, you are only limited by your imagination!

In [ ]:
bands_plot = bands_plot.update_inputs(bands_style={})

## Displaying the smallest gaps

The easiest thing to do is to let `BandsPlot` discover where the (minimum) gaps are.

This is indicated by setting the `gap` parameter to `True`. One can also use `gap_color` if a particular color is desired.

In [ ]:
bands_plot.update_inputs(gap=True, gap_color="green", Erange=[-10, 10])

This displays the minimum gaps. However there may be some issues with it: it will show **all** gaps with the minimum value. That is, if you have repeated points in the brillouin zone it will display multiple gaps that are equivalent. 

What's worse, if the region where your gap is is very flat, two consecutive points might have the same energy. Multiple gaps will be displayed one glued to another.

To help cope with this issues, you have the `direct_gaps_only` and `gap_tol`.

In this case, since we have no direct gaps, setting `direct_gaps_only` will hide them all:

In [ ]:
bands_plot.update_inputs(direct_gaps_only=True)

This example is not meaningful for `gap_tol`, but it is illustrative of what `gap_tol` does. It is the **minimum k-distance between two points to consider them "the same point"** in the sense that only one of them will be used to show the gap. In this case, if we set `gap_tol` all the way up to 3, the plot will consider the two gamma points to be part of the same "point" and therefore it will only show the gap once.

In [ ]:
bands_plot.update_inputs(direct_gaps_only=False, gap_tol=3)

This is not what `gap_tol` is meant for, since it is thought to remediate the effect of locally flat bands, but still you can get the idea of what it does.

In [ ]:
bands_plot = bands_plot.update_inputs(gap=False, gap_tol=0.01)

## Displaying custom gaps

If you are not happy with the gaps that the plot is displaying for you or **you simply want gaps that are not the smallest ones**, you can always use `custom_gaps`.

Custom gaps should be a list where each item specifies how to draw that given gap. The key labels of each item are `from` and `to`, which specifies the k-points through which you want to draw the gap. The rest of labels are the typical styling labels: `color`, `width`...

For example, if we want to plot the gamma-gamma gap:

In [ ]:
bands_plot.update_inputs(custom_gaps=[{"from": "Gamma", "to": "Gamma", "color": "red"}])

Notice how we got the gap probably not where we wanted, since it would be better to have it in the middle `Gamma` point, which is more visible. Instead of the K point name, you can also pass the K value.

Now, you'll be happy to know that you can easily access the k values of all labels, as they are stored as part of the attributes of the `k` coordinate in the bands dataarray:

In [ ]:
bands_plot.nodes["bands_data"].get().k.axis

Now all we need to do is to grab the value for the second gamma point:

In [ ]:
axis_info = bands_plot.nodes["bands_data"].get().k.axis

gap_k = None
for val, label in zip(axis_info["tickvals"], axis_info["ticktext"]):
    if label == "Gamma":
        gap_k = val
gap_k

And use it to build a custom gap:

In [ ]:
bands_plot.update_inputs(custom_gaps=[{"from": gap_k, "to": gap_k, "color": "orange"}])

## Displaying spin texture

If your bands plot comes from a non-colinear spin calculation (or is using a `Hamiltonian` with non-colinear spin), you can pass `"x"`, `"y"` or `"z"` to the `spin` setting in order to get a display of the spin texture.

Let's read in a hamiltonian coming from a spin orbit SIESTA calculation, which is obtained from [this fantastic spin texture tutorial](https://github.com/juijan/TopoToolsSiesta/tree/master/Tutorials/Exercise/TI_02):

In [ ]:
import sisl

siesta_files = sisl.get_environ_variable("SISL_FILES_TESTS") / "siesta"

In [ ]:
H = sisl.get_sile(siesta_files / "Bi_hexagonal" / "Bi_hexagonal.fdf").read_hamiltonian()
H.spin.is_spinorbit

Generate the path for our band structure:

In [ ]:
band_struct = sisl.BandStructure(
    H,
    points=[
        [1.0 / 2, 0.0, 0.0],
        [0.0, 0.0, 0.0],
        [1.0 / 3, 1.0 / 3, 0.0],
        [1.0 / 2, 0.0, 0.0],
    ],
    divisions=301,
    names=["M", r"Gamma", "K", "M"],
)

And finally generate the plot:

In [ ]:
spin_texture_plot = band_struct.plot.bands(Erange=[-2, 2])
spin_texture_plot

Now it's time to add spin texture to these bands. Remember the section on styling bands? If you haven't checked it, take a quick look at it, because it will come handy now. The main point to take from that section for our purpose here is that each key in the styles accepts a callable.

As in other cases through the `sisl.viz` module, we provide callables that will work out of the box for the most common styling. In this case, what we need is the `SpinMoment` node. We will import it and use it simply by specifying the axis.

In [ ]:
from sisl.viz.data_sources import SpinMoment

spin_texture_plot.update_inputs(bands_style={"color": SpinMoment("x"), "width": 3})

# We hide the legend so that the colorbar can be easily seen.
spin_texture_plot.update_layout(showlegend=False)

There is nothing magic about the `SpinMoment` node. If you pass a dummy callable as we did in the styling section, you will see that the bands data now contains a `spin_moments` variable since it comes from a non-colinear calculation. It is just a matter of grabbing that variable:

In [ ]:
def color(data):
    """Dummy function to see what we receive."""
    print(data)
    return "green"


spin_texture_plot.update_inputs(bands_style={"color": color})

Note that, as shown in the styling section, you can use the `colorscale` input to change the colorscale, or use the `SpinMoment` node for the other styling keys. For example, we can set the width of the band to display whether there is some spin moment, and the color can show the sign.

In [ ]:
spin_texture_plot.update_inputs(
    bands_style={"color": SpinMoment("x"), "width": abs(SpinMoment("x")) * 40}
).update_layout(showlegend=False).show("png")

Notice how we did some postprocessing to adapt the values of the spin moment to some number that is suitable for the width. This is possible thanks to the magic of nodes!

We hope you enjoyed what you learned!